## Import Libraries

In [2]:
# from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader

In [4]:
import llama_index

In [3]:
from llama_index import core

In [2]:
dir(llama_index)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [21]:
dir(llama_index.core)

['BaseCallbackHandler',
 'BasePromptTemplate',
 'Callable',
 'ChatPromptTemplate',
 'ComposableGraph',
 'Document',
 'DocumentSummaryIndex',
 'GPTDocumentSummaryIndex',
 'GPTKeywordTableIndex',
 'GPTListIndex',
 'GPTRAKEKeywordTableIndex',
 'GPTSimpleKeywordTableIndex',
 'GPTTreeIndex',
 'GPTVectorStoreIndex',
 'IndexStructType',
 'KeywordTableIndex',
 'KnowledgeGraphIndex',
 'ListIndex',
 'MockEmbedding',
 'NullHandler',
 'Optional',
 'Prompt',
 'PromptHelper',
 'PromptTemplate',
 'QueryBundle',
 'RAKEKeywordTableIndex',
 'Response',
 'SQLContextBuilder',
 'SQLDatabase',
 'SQLDocumentContextBuilder',
 'SelectorPromptTemplate',
 'ServiceContext',
 'Settings',
 'SimpleDirectoryReader',
 'SimpleKeywordTableIndex',
 'StorageContext',
 'SummaryIndex',
 'TreeIndex',
 'VectorStoreIndex',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'async_utils',
 'base',
 'bridg

In [23]:
llama_index.llms.HuggingFaceLLM

AttributeError: module 'llama_index.llms' has no attribute 'HuggingFaceLLM'

In [15]:
dir(llama_index.core.llms.CustomLLM)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '__weakref__',
 '_abc_impl',
 '_as_query_component',
 '_calculate_keys',
 '_copy_and_s

In [3]:
import llama_index

ImportError: cannot import name 'BaseCache' from 'langchain.cache' (/home/paperspace/Repos/RAG-App-Example/venv/lib/python3.11/site-packages/langchain/cache.py)

In [7]:
import inspect

# inspect.getmembers(llama_index)
# import sys

for name, obj in inspect.getmembers(llama_index):
        if inspect.isclass(obj):
            print(obj)

<class 'llama_index.callbacks.base_handler.BaseCallbackHandler'>
<class 'llama_index.prompts.base.BasePromptTemplate'>
<class 'llama_index.prompts.base.ChatPromptTemplate'>
<class 'llama_index.indices.composability.graph.ComposableGraph'>
<class 'llama_index.schema.Document'>
<class 'llama_index.indices.document_summary.base.DocumentSummaryIndex'>
<class 'llama_index.indices.document_summary.base.DocumentSummaryIndex'>
<class 'llama_index.indices.keyword_table.base.KeywordTableIndex'>
<class 'llama_index.indices.knowledge_graph.base.KnowledgeGraphIndex'>
<class 'llama_index.indices.list.base.SummaryIndex'>
<class 'llama_index.indices.keyword_table.rake_base.RAKEKeywordTableIndex'>
<class 'llama_index.indices.keyword_table.simple_base.SimpleKeywordTableIndex'>
<class 'llama_index.indices.tree.base.TreeIndex'>
<class 'llama_index.indices.vector_store.base.VectorStoreIndex'>
<enum 'IndexStructType'>
<class 'llama_index.indices.keyword_table.base.KeywordTableIndex'>
<class 'llama_index.ind

In [ ]:
import sys
current_module = sys.modules[llama_index]

KeyError: <module 'llama_index' (<_frozen_importlib_external.NamespaceLoader object at 0x7f3b6f81a990>)>

In [13]:
import torch
from transformers import BitsAndBytesConfig
from llama_index import VectorStoreIndex, StorageContext, Document, ServiceContext, PromptTemplate
from llama_index.prompts import PromptTemplate
# from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms import HuggingFaceLLM
# from llama_index.response.notebook_utils import display_response
from llama_index.vector_stores import PineconeVectorStore
from pinecone import Pinecone, PodSpec
from datasets import load_dataset
import os

## Create Hugging Face LLM

In [14]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [15]:
def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}\n"
# ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt

In [16]:
llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<|system|>\n\n<|user|>\n{query_str}\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.3, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",

)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.26s/it]


In [5]:
# service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

In [6]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("WhereIsAI/UAE-Large-V1")
# model = AutoModel.from_pretrained("WhereIsAI/UAE-Large-V1")

## Create Pinecone VectorDB Index

In [17]:
os.environ['PINECONE_API_KEY'] = 'ad2c4c6e-833e-4ef5-b9b1-eeca2b976564'

In [18]:
pc = Pinecone(
    api_key = os.getenv('PINECONE_API_KEY')
    )

try:
    pc.create_index(
        name='my-index',
        dimension=1024,
        metric='euclidean',
        spec=PodSpec(
            replicas= 1, 
            shards= 1, 
            pod_type="p1",
            environment='gcp-starter'
        )        
    )
    print('Index created')
except:
    print('Index already exists')

pinecone_index = pc.Index("my-index")

Index already exists


### Load Dataset

In [22]:
dataset = load_dataset("neural-bridge/rag-dataset-12000")

questions = dataset['train']['question']
contexts = dataset['train']['context']
answers = dataset['train']['answer']

# Remove records where question is None
indices_to_pop = [i for i, value in enumerate(questions) if value is None]
indices_to_pop.sort(reverse=True)

for index in indices_to_pop:
    questions.pop(index)
    contexts.pop(index)
    answers.pop(index)

documents = [Document(text=q, metadata={"context": c}) for q, c in zip(questions, contexts)]

### Store Embeddings in Pinecone

In [11]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    add_sparse_vector=True,
)

# vector_store = PineconeVectorStore(pinecone.Index("quickstart"))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, llm=None)
# index = VectorStoreIndex.from_vector_store(vector_store=vector_store, llm=llm, embed_model="local:WhereIsAI/UAE-Large-V1", chunk_size=2048)
# llm=llm, embed_model="local:WhereIsAI/UAE-Large-V1", chunk_size=2048)

ImportError: cannot import name 'global_handler' from 'llama_index.core' (/home/paperspace/Repos/RAG-App-Example/venv/lib/python3.11/site-packages/llama_index/core/__init__.py)

In [15]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    add_sparse_vector=True,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:WhereIsAI/UAE-Large-V1", chunk_size=2048)

In [17]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [16]:
index = GPTVectorStoreIndex(
    storage_context=storage_context, service_context=service_context
)

ValueError: One of nodes, objects, or index_struct must be provided.

In [ ]:
VectorStoreIndex.from_vector_store()

In [21]:
dir(VectorStoreIndex)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_nodes_to_index',
 '_aget_node_with_embedding',
 '_async_add_nodes_to_index',
 '_build_index_from_nodes',
 '_delete_node',
 '_get_node_with_embedding',
 '_insert',
 '_is_protocol',
 'as_chat_engine',
 'as_query_engine',
 'as_retriever',
 'build_index_from_nodes',
 'delete',
 'delete_nodes',
 'delete_ref_doc',
 'docstore',
 'from_documents',
 'from_vector_store',
 'index_id',
 'index_struct',
 'index_struct_cls',
 'insert',
 'insert_nodes',
 'ref_doc_info',
 'refresh',
 '

In [26]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    add_sparse_vector=True,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:WhereIsAI/UAE-Large-V1", chunk_size=2048)

index = VectorStoreIndex.from_documents(
    [], storage_context=storage_context, service_context=service_context
)

query_engine = index.as_query_engine()

## Document Retrieval

In [27]:
# query_engine = index.as_query_engine()
response = query_engine.query("Who gave the author the pasta salad recipe?")
# response = query_engine.query("What are some of the features of Fabiana Filippi's shirts and blouses?")
print(response.response)

Ann Marie gave the author the pasta salad recipe.


In [12]:
import llama_index

dir(llama_index.core.llms.llm)

['Any',
 'BaseLLM',
 'BaseLLMComponent',
 'BaseModel',
 'BaseOutputParser',
 'BasePromptTemplate',
 'CBEventType',
 'Callable',
 'ChainMap',
 'ChatMessage',
 'ChatResponseAsyncGen',
 'ChatResponseGen',
 'CompletionResponseAsyncGen',
 'CompletionResponseGen',
 'CompletionToPromptType',
 'Dict',
 'EventPayload',
 'Field',
 'InputKeys',
 'LLM',
 'LLMChatComponent',
 'LLMCompleteComponent',
 'List',
 'MessageRole',
 'MessagesToPromptType',
 'Optional',
 'OutputKeys',
 'PromptTemplate',
 'Protocol',
 'PydanticProgramMode',
 'QueryComponent',
 'Sequence',
 'StringableInput',
 'TokenAsyncGen',
 'TokenGen',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'astream_chat_response_to_tokens',
 'astream_completion_response_to_tokens',
 'default_completion_to_prompt',
 'generic_messages_to_prompt',
 'get_args',
 'prompt_to_messages',
 'root_validator',
 'runtime_checkable',
 'stream_chat_response_to_tokens',
 'stream_completion_respo

In [28]:
import random

rand_num = random.randrange(len(questions))
rand_num = 1345
print(f"index: {rand_num}")
print(questions[rand_num])
# print(contexts[rand_num])
print(answers[rand_num])

index: 1345
Who gave the author the pasta salad recipe?
Ann Marie gave the author the pasta salad recipe.


In [ ]:
# Other areas for improvement on RAGs,
# Embed the context instead of questions
# Deploy using TGIF and handle it in a basic front end (Next JS with a simple simple read out and search box)
# Just use Postman for API showing and streaming. How to do that with FastAPI app

In [ ]:
## Then create prompt for LLM

In [59]:
questions[5]

"What are some of the features of Fabiana Filippi's shirts and blouses?"

In [64]:
contexts[5]

'The prices indicated in this catalogue do not include shipping costs.\nFabiana Filippi\'s shirts and blouses are the perfect base to create solid modern and refined looks thanks to their innate ability to be easily matched to different shapes, fabrics and colours. "Must-have" items in the wardrobe, always able to confer impeccable elegance. Silk and cotton shirts, carefully-selected precious fabrics are the protagonists of a timeless style and ultra-versatile use that satisfies the needs of multiple daily commitments. Elegant women’s shirts, interpreted with classic, slim and over cuts, in ultra-modern style. Short, long and ¾ sleeve blouses and shirts, available in different lengths up to maxi lines. Men’s cut, Korean, V, round necks are examples that underscore the versatility of the various models to wear during the day and at night, to feel the best yarns on the skin and embellish the look with surprising style details. The union between feminine essence and everyday versatility r

In [7]:
from datasets import load_dataset

dataset = load_dataset("neural-bridge/rag-dataset-12000")

Generating test split: 100%|██████████| 2400/2400 [00:00<00:00, 56947.53 examples/s]


In [10]:
hi = [('1', '2'), ('3', '4')]
for a, b in hi:
    print(a)
    print(b)

1
2
3
4


In [21]:
from llama_index import Document

text_list1 = ["a", "text1"]
text_list2 = ["b", "text2"]
documents = [Document(text=t, metadata={'hello': t2}) for t, t2 in zip(text_list1, text_list2)]

In [24]:
None in contexts

False

In [27]:
indices_to_pop = [i for i, value in enumerate(questions) if value is None]

In [28]:
indices_to_pop

[340, 2818]

In [29]:
indices_to_pop.sort(reverse=True)

# Pop elements
for index in indices_to_pop:
    questions.pop(index)
    contexts.pop(index)

In [30]:
from llama_index import Document

# questions = dataset['train']['question']
# contexts = dataset['train']['context']
documents = [Document(text=q, metadata={"context": c}) for q, c in zip(questions, contexts)]

In [ ]:
document = Document(
    text="text",
    metadata={"filename": "<doc_file_name>", "category": "<category>"},
)

In [ ]:
from llama_index.schema import TextNode

node1 = TextNode(text="<text_chunk>", id_="<node_id>")
node2 = TextNode(text="<text_chunk>", id_="<node_id>")
nodes = [node1, node2]
index = VectorStoreIndex(nodes)

In [ ]:
# documents = SimpleDirectoryReader(
#     "../../examples/data/paul_graham"
# ).load_data()

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)